# Homework 3

## FINM 37400 - 2025

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

#### HBS Case: Fixed-Income Arbitrage in a Financial Crisis (C): Spread and Swap Spread in November 2008


***

In [53]:
import datetime
from datetime import datetime as dt
from datetime import timedelta
from typing import Optional, Union, List, Tuple, Dict
import warnings

import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
FutureWarning

import numpy as np
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from scipy.optimize import fsolve
from scipy import interpolate
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
%matplotlib inline

from scheffer_quant.fixed_income.fi_pricing import *
from scheffer_quant.fixed_income.treasury import *
from scheffer_quant.fixed_income.curve_construction import *
from scheffer_quant.fixed_income.fi_cash_flows import *
from scheffer_quant.strategies.bond_pair_trading import *
from scheffer_quant.portfolio.analysis import *

pd.options.display.max_columns = 30
pd.options.display.max_colwidth = 100
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# 1 HBS Case: Fixed-Income Arbitrage in a Financial Crisis (C): Spread and Swap Spread in November 2008

In [30]:
QUOTE_DATE = '2024-10-31'
treasury_file_path = f'../data/treasury_ts_duration_{QUOTE_DATE}.xlsx'
Yields_file_path = '../data/yields.xlsx'

yields = pd.read_excel(Yields_file_path, sheet_name='yields').set_index('caldt')

data = pd.read_excel(treasury_file_path,sheet_name='database')
price = pd.read_excel(treasury_file_path,sheet_name='price').set_index('quote date')
duration = pd.read_excel(treasury_file_path,sheet_name='duration').set_index('quote date')

data_info =  data.drop_duplicates(subset='KYTREASNO', keep='first').set_index('KYTREASNO')
data_info[['type','issue date','maturity date','cpn rate']]

,type,issue date,maturity date,cpn rate
KYTREASNO,,,,
207391,note,2019-08-15,2029-08-15,1.6250
207392,bond,2019-08-15,2049-08-15,2.2500


## Simplification of the setup

The date is Nov 4, 2008.

**Treasury bond**
* Suppose the Treasury bond matures exactly 30 years later, on Nov 4, 2038 rather than May 15, 2008. 
* The YTM of this freshly issued treasury is 4.193\% with a semiannual coupon of 4.50\%, same as is given in the case. (So we're just changing the maturity date to simplify things, but keeping the market data.)

**Swap**
* The fixed leg of the swap pays semiannually, with swap rate of 4.2560\%, as given in the case.
* The floating leg of the swap also pays semiannually--not quarterly--such that the payment dates are identical on both legs. Thus, it also resets the floating rate semiannually, not quarterly.
* The floating rate of the swap equals the repo rate used in the trade. Thus, these two rates cancel in the financing of the trade. (No need to consider the TED spread.) 

**Notes from class:**
- Swap spread is negative:
    - Set up a trade you have a positive payment:
    - Get fixed rate on the treasuries and pay a floating repo rate
    - Pay lower fixed rate in the swap and received floating rate (used to pay the repo)
    - A trade for 100 notional on the bond and 100 on the swap, you get half of the spread * 100 per semiannual period.
- If you want to hold the swap spread until maturity, then the trade must be made on the YTM, because if the coupon rate is higher than the YTM, then the price of the treasury is higher than 100 but it is 100% certain to converge down to 100.
- Also, even if the coupon rate in the treasuries is higher than the swap rate, because you matched by duration and bought less swap, you receive less coupons.

## Case Clarifications


### Duration Quotes
Bond
* Quote: Val01 of bond is .1746 per bp per $1 face value
* Class terminology: Modified dollar duration is .1746 per $100 face value

Swap
* Quote: DV01 of swap is 1.7mm per 1 billion notional.
* Class terminology: Modified dollar duration is 100(1.7/1000) per $100 face value.

Thus, modified dollar duration for each per 100 face is
* Bond = .1746
* Swap = .1700

### Hedge Ratio

In figuring out the hedge ratio, they set up the hedge per dollar of face value. 

    *so Mills would need to buy face amount $0.97 billion*
    
No, this hedge should be for market value, not face amount given that the case is already using **modified** duration which includes the dirty price.
    

### Maturity Mismatch

The maturity of the bond is August 2038, whereas the date is Nov 2008. Thus, the bond has less than 30 years to maturity, yet he is entering a 30-year swap. 

For simplicity, we imagine the bond is issued in Nov 2008 and thus has maturity of 30 years at the time of the case.

However, then the case quotes for the Nov price and YTM of the bond no longer are accurate. Use one and adjust the other. Namely, we could...
    * use the Nov 4 **YTM** quoted in the case, and re-adjust the the bond.
    * use the Nov 4 **price** quoted in the case, and re-adjust the YTM.
    
We do the former, keep the quoted YTM, assume time-to-maturity of `30`, and recalculate the Nov 2008 price. (It is close to the quoted price in the case.)

In [31]:
# Parameters
YTM = [0.04193, .0436] # [Initial, After Market Movement]
CPNRATE = [0.0450, np.nan] # [Fixed rate, floating rate]
SWAPRATE = [.042560, .0408] # [Initial, After Market Movement]

ORIGINAL_PRICE_0 = 105
PAR = 100

NOTIONAL = 500e6
SET_SIZE = 'swap'

HAIRCUT = .02
SWAP_FACE = 100

DELTATIME = .5
tau0 = 30 # maturity in years
tau1 = tau0-DELTATIME

In [32]:
'''
# Adjustements (Optional)
USE_CPN = True
USE_YTM = False
USE_PRICE = False

if not USE_CPN:
    CPNRATE[0] = YTM[0]

if USE_YTM:
    if USE_PRICE:
        warning('Case YTM and Price are inconsistent; recalculating price')
        ORIGINAL_PRICE_0 = bond_pricer(ttm=tau0,ytm=YTM[0],cpn_rate=CPNRATE[0])

else:
    if USE_PRICE:  
        YTM[0] = calculate_ytm(price=ORIGINAL_PRICE_0, ttm=tau0, cpn_rate=CPNRATE[0])
    else:
        YTM[0] = CPNRATE[0]
        ORIGINAL_PRICE = PAR
'''

"\n# Adjustements (Optional)\nUSE_CPN = True\nUSE_YTM = False\nUSE_PRICE = False\n\nif not USE_CPN:\n    CPNRATE[0] = YTM[0]\n\nif USE_YTM:\n    if USE_PRICE:\n        warning('Case YTM and Price are inconsistent; recalculating price')\n        ORIGINAL_PRICE_0 = bond_pricer(ttm=tau0,ytm=YTM[0],cpn_rate=CPNRATE[0])\n\nelse:\n    if USE_PRICE:  \n        YTM[0] = calculate_ytm(price=ORIGINAL_PRICE_0, ttm=tau0, cpn_rate=CPNRATE[0])\n    else:\n        YTM[0] = CPNRATE[0]\n        ORIGINAL_PRICE = PAR\n"

## 1.0.

Report the price of the 30-year T-bond in Nov 2008. Given the discussion about `Maturity Mismatch`, we are repricing the bond, so it will not exactly equal `105` as reported in the case.

In [33]:
PRICE_0 = bond_pricer(ttm=tau0,ytm=YTM[0],cpn_rate=CPNRATE[0])
PRICE_1 = bond_pricer(ttm=tau1,ytm=YTM[1],cpn_rate=CPNRATE[0])

summary = pd.DataFrame(index=['tenor', 'coupon rate','price','YTM','swap rate'],columns = ['Nov 2008', 'May 2009'],dtype=float)
summary.loc['tenor'] = [tau0, tau1]
summary.loc['coupon rate'] = [CPNRATE[0], CPNRATE[0]]
summary.loc['YTM'] = YTM
summary.loc['price'] = [PRICE_0, PRICE_1]
summary.loc['swap rate'] = SWAPRATE
summary.loc['spread'] = summary.loc['swap rate'] - summary.loc['YTM']
#summary.loc['coupon spread'] = summary.loc['swap rate'] - summary.loc['coupon rate']

summary.T.style.format({'tenor':'{:.1f}', 'price':'{:.2f}', 'coupon rate':'{:.2%}', 'YTM':'{:.2%}', 'swap rate':'{:.2%}', 'spread':'{:.2%}'})

,tenor,coupon rate,price,YTM,swap rate,spread
Nov 2008,30.0,4.50%,105.21,4.19%,4.26%,0.06%
May 2009,29.5,4.50%,102.31,4.36%,4.08%,-0.28%


## 1.1

List the projected cashflows on May 4, 2009, exactly six months into the trade, on the first coupon and swap date.

#### Clarification
List these cashflows for face value of $1B, not the $0.97B noted in the case. As mentioned in "Case Clarifications", we will not use this number. Rather, we calculate our own hedge ratio in a problem below.

In [34]:
SOFR = np.nan

In [35]:
CF = pd.DataFrame(index=['T bond','Repo','Swap (floating leg)','Swap (fixed leg)'],columns=['May 2009'],dtype=float)
CF.loc['Repo'] - SOFR
CF.loc['Swap (floating leg)'] = SOFR
CF.loc[['T bond']] = PAR * CPNRATE[0] /2
CF.loc[['Swap (fixed leg)']] = -PAR * SWAPRATE[0]/2
CF.loc['Net Payment'] = CF.sum(axis=0)
CF.style.format('${:,.2f}')

,May 2009
T bond,$2.25
Repo,$nan
Swap (floating leg),$nan
Swap (fixed leg),$-2.13
Net Payment,$0.12


#### SOFR rate cancels

No need to account for the repo rate or the swap's floating payment, as they both are modeled in this problem with SOFR, and thus net to zero.

## 1.2

What is the duration of...
* the T-bond
* the swap

Remember that...
* the swap can be decomposed into a fixed-rate bond and a floating-rate note
* a floating-rate note has duration equal to the time until the next reset. Thus, at initialization, it has duration equal to 0.5 years.

Is the duration for the "paying-fixed" swap positive or negative? Is it bigger or smaller in magnitude than the T-bond?

For this problem, calculate the Macauley duration and the dollar (Macauley) duration.

In [36]:
tab_duration = pd.DataFrame(dtype=float, index=['T bond','fixed leg', 'floating leg'], columns=['duration'])
tab_duration.loc['T bond'] = duration_closed_formula(ttm=tau0, ytm=summary.loc['YTM','Nov 2008'], cpn_rate=summary.loc['coupon rate','Nov 2008'])
tab_duration.loc['fixed leg'] = duration_closed_formula(ttm=tau0, ytm=summary.loc['swap rate','Nov 2008'])
tab_duration.loc['floating leg'] = .5
tab_duration.loc['repo'] = .5
prices = np.array([PRICE_0, PAR, PAR, PRICE_0 * (1-HAIRCUT)])
tab_duration['dollar duration'] = tab_duration['duration'] * prices

tab_net = pd.DataFrame(dtype=float, index=['T bond repo','swap'], columns=['duration','dollar duration'])
tab_net.loc['T bond repo'] = tab_duration.loc['T bond'] - tab_duration.loc['repo']
tab_net.loc['swap'] = tab_duration.loc['fixed leg'] - tab_duration.loc['floating leg']
tab_net.loc['net'] = tab_net.loc['T bond repo'] - tab_net.loc['swap']

display(tab_duration)

display(tab_net)

,duration,dollar duration
T bond,17.0836,1797.4251
fixed leg,17.2127,1721.2744
floating leg,0.5000,50.0000
repo,0.5000,51.5545


,duration,dollar duration
T bond repo,16.5836,1745.8706
swap,16.7127,1671.2744
net,-0.1291,74.5961


## 1.3

What hedge ratio should be used to balance the notional size of the Treasury bond with the notional size of the swap, such that it is a duration-neutral position?

Specifically, if the trader enters the swap paying fixed on \$500 million notional, how large of a position should they take in the Treasury bond?

In [37]:
hedge_ratio = -tab_net.loc['swap','dollar duration'] / tab_net.loc['T bond repo','dollar duration']
 
if SET_SIZE == 'Tbond':
    notional_tbond = NOTIONAL
else:
    if SET_SIZE != 'swap':
        warning('Sizing is not specified; fixing swap notional')
    notional_tbond = -NOTIONAL * hedge_ratio

notional_swaps = notional_tbond * (1/hedge_ratio)
n_tbonds = notional_tbond / PRICE_0
n_swaps = notional_swaps / SWAP_FACE

contracts = pd.DataFrame(np.array([n_tbonds, n_swaps]), index=['T bond repo','swap'], columns=['positions'])
contracts

,positions
T bond repo,4549201.3611
swap,-5000000.0000


## 1.4

Suppose it is May 4, 2009, exactly six months after putting the trade on.

The spread is at -28 bps due to...
* The YTM on a new 30-year bond has risen to 4.36\%
* The swap rate on a new 30-year swap has dropped to 4.08\%

Explain conceptually how this movement impacts the components of the trade.

In [38]:
summary.T.style.format({'tenor':'{:.1f}', 'price':'{:.2f}', 'coupon rate':'{:.2%}', 'YTM':'{:.2%}', 'swap rate':'{:.2%}', 'spread':'{:.2%}'})

,tenor,coupon rate,price,YTM,swap rate,spread
Nov 2008,30.0,4.50%,105.21,4.19%,4.26%,0.06%
May 2009,29.5,4.50%,102.31,4.36%,4.08%,-0.28%


## 1.5

Calculate the value of the position on May 4, 2009, immediately after the first coupon and swap payments and swap reset. 

* Calculate the revised price of the Treasury bond by assuming you can apply the (May 4) 30-year YTM as a discount rate to the 29.5 year bond. (We are just using this for a rough approximation. You know that good pricing would require a discount curve, but let's not get bogged down with that here.)


* Calculate the value of the swap by decomposing it into a fixed-rate bond and a floating-rate bond.
    * The 29.5 year fixed-rate leg is priced using the (May 4) 30-year swap rate as a discount rate.
    * The floating-rate leg is priced at par given that floating-rate notes are par immediately after resets.
    
**Note**

You are being asked to calculate these valuations using the exact formula between price, cashflows, and YTM discount rate. We are not simply approximating with duration, as we already know the position was set up with zero dollar duration.

From the Discussion notebook, we have this formula expressing a bond's price as a function of the coupon, $c$, and the YTM, $y_j$.

$
P_j(t,T,c) = \sum_{i=1}^{n-1}\frac{100\left(\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T_i-t)}} + \frac{100\left(1+\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T-t)}}
$

In [40]:
prices = pd.DataFrame(index=['T bond', 'swap'],dtype=float,columns=['Nov 2008'])

prices.loc['T bond','Nov 2008'] = PRICE_0
prices.loc['swap','Nov 2008'] = 0

prices.loc['T bond','May 2009'] = PRICE_1
prices.loc['swap','May 2009'] = bond_pricer(ttm=tau1, ytm=summary.loc['swap rate','May 2009'], cpn_rate=summary.loc['swap rate','Nov 2008']) - PAR

prices.T.style.format('${:,.2f}')

,T bond,swap
Nov 2008,$105.21,$0.00
May 2009,$102.31,$3.00


The rising YTM on the T bond suggests...
* lower (adjusted) price for the T-bond.
* we are long the T-bond, so this would be a loss.

The lower swap rate suggests...
* higher value on the fixed leg of a swap.
* we are paying fixed, (ie short the fixed leg), so this would again be a loss.

The floating leg of the swap and repo rate cancel.

## 1.6

Accounting for the change in value of the positions, as well as the 6-month cashflows paid on May 4, 
* what is the net profit and loss (pnl) of the position?
* what is the return on the equity capital, considering that there was a 2\% haircut (equity contribution) on the size of the initial treasury bond position.

In [42]:
CF

,May 2009
T bond,2.2500
Repo,NaN
Swap (floating leg),NaN
Swap (fixed leg),-2.1280
Net Payment,0.1220


In [43]:
pnl=pd.DataFrame(dtype=float,index=['T bond','swap'],columns=['cashflow'])

pnl['cashflow'] = CF.loc[['T bond','Swap (fixed leg)']].values * contracts.abs().values
pnl['capital gains'] = prices.diff(axis=1)['May 2009'].values * contracts.values[:,0]

pnl.loc['net'] = pnl.sum()

pnl['total'] = pnl.sum(axis=1)

pnl.style.format('${:,.2f}',na_rep='')

,cashflow,capital gains,total
T bond,"$10,235,703.06","$-13,201,241.29","$-2,965,538.23"
swap,"$-10,640,000.00","$-15,016,747.03","$-25,656,747.03"
net,"$-404,296.94","$-28,217,988.32","$-28,622,285.26"


In [46]:
capital = pd.DataFrame(prices.iloc[:,0].values * contracts.values[:,0], index=['T bond','swap'],columns=['assets_0'])
capital['equity_0'] = capital['assets_0'] * HAIRCUT
capital.loc['net'] = capital.sum()

capital['pnl'] = pnl['total']
capital['return'] = capital['pnl']/capital['equity_0']
capital.loc[['T bond','swap'],'return'] = np.nan

capital.style.format({'assets_0':'${:,.2f}','equity_0':'${:,.2f}','pnl':'${:,.2f}','return':'{:.2%}'},na_rep='')

,assets_0,equity_0,pnl,return
T bond,"$478,636,404.56","$9,572,728.09","$-2,965,538.23",
swap,$-0.00,$-0.00,"$-25,656,747.03",
net,"$478,636,404.56","$9,572,728.09","$-28,622,285.26",-299.00%


***

# 2. Factor Duration

### Data

This problem uses data from,
* `/data/yields.xlsx`
* `/data/treasury_ts_duration_2024-10-31.xlsx`

#### Load Yields

In [47]:
filepath = '../data/yields.xlsx'
yields = pd.read_excel(filepath, sheet_name='yields')
yields.set_index('caldt',inplace=True)

#### Load Prices and Durations of Two Treasuries

In [48]:
QUOTE_DATE = '2024-10-31'
filepath = f'../data/treasury_ts_duration_{QUOTE_DATE}.xlsx'

data = pd.read_excel(filepath,sheet_name='database')
data_info =  data.drop_duplicates(subset='KYTREASNO', keep='first').set_index('KYTREASNO')
data_info[['type','issue date','maturity date','cpn rate']]

,type,issue date,maturity date,cpn rate
KYTREASNO,,,,
207391,note,2019-08-15,2029-08-15,1.6250
207392,bond,2019-08-15,2049-08-15,2.2500


You will largely focus on the sheets which give the timeseries of prices and durations for each of the two securities, as shown in the following code.

In [49]:
SHEET_PRICE = 'price'
SHEET_DURATION = 'duration'
INDEX_NAME = 'quote date'

price = pd.read_excel(filepath,sheet_name=SHEET_PRICE).set_index(INDEX_NAME)
duration = pd.read_excel(filepath,sheet_name=SHEET_DURATION).set_index(INDEX_NAME)

### 2.1.

Construct the following yield-curve factors from the `yields` data set:

$\begin{align}
x^{\text{level}}_t =& \frac{1}{N_{\text{yields}}}\sum_{i=1}^{N_{\text{yields}}} y^{(i)}_t\\
x^{\text{slope}}_t =& y^{(30)}_t - y^{(1)}_t\\
x^{\text{curvature}}_t =& -y^{(1)}_t + 2 y^{(10)}_t - y^{(30)}_t
\end{align}$

In [50]:
yields.index.name = 'date'
shapes = pd.DataFrame(yields.mean(axis=1), columns=['level'])
shapes['slope'] = yields[30] - yields[1]
shapes['curvature'] = -yields[1] + 2* yields[10] - yields[30]
shapes


,level,slope,curvature
date,,,
1961-06-14,3.5910,0.8793,0.8866
1961-06-15,3.6147,0.8934,0.9667
1961-06-16,3.6174,0.9001,0.9666
1961-06-19,3.6306,0.9167,1.0140
1961-06-20,3.6273,0.8926,0.9756
...,...,...,...
2024-12-24,4.5023,0.4496,0.0865
2024-12-26,4.4911,0.4669,0.0881
2024-12-27,4.5154,0.5436,0.1518


### 2.2

Get the bond prices and durations for the two bonds in the data set referenced above.

#### Align the data

Align the bond pricing data with the yield factor data, so that you have data for both in the intersection of their dates.


#### Estimate the regression

Estimate the regression in the form of day-over-day differences for both bond prices and factors. That is, we are using regression to approximate the factor duration equation,

$\begin{align}
\frac{dP}{P} = \alpha + \beta_L dx_{\text{level}} + \beta_S dx_{\text{slope}} + \beta_C dx_{\text{curvature}} + \epsilon
\end{align}$

Report the betas for each of these factors, for each of the bond prices.

In [51]:
merged_price_shape = pd.merge(price, shapes, left_index=True, right_index=True, how='inner')
merged_price_shape

,207391,207392,level,slope,curvature
2019-08-09,98.8828,99.7891,1.8064,0.4162,-0.6214
2019-08-12,99.7969,102.5547,1.7156,0.3490,-0.6385
2019-08-13,99.2812,101.8672,1.7833,0.2816,-0.6696
2019-08-14,100.4062,105.1797,1.6781,0.1980,-0.6965
2019-08-15,100.8828,106.2344,1.6245,0.2172,-0.6901
...,...,...,...,...,...
2024-11-22,88.7402,63.7227,4.4182,0.1321,-0.2379
2024-11-25,89.2871,65.3789,4.2948,0.0222,-0.3536
2024-11-26,89.2148,65.1758,4.3138,0.0240,-0.3131
2024-11-27,89.4375,65.6562,4.2738,0.0128,-0.3493


In [57]:
from sklearn.linear_model import LinearRegression

dp_pct = price.pct_change().dropna()
dfac = shapes.diff().dropna()
dp_pct, dfac = dp_pct.align(dfac,join='inner',axis=0)

factor_durations = pd.DataFrame(-LinearRegression(fit_intercept=True).fit(dfac,dp_pct).coef_.T, columns = dp_pct.columns, index= dfac.columns)
factor_durations

,207391,207392
level,0.0699,0.1980
slope,0.0048,0.1265
curvature,0.0109,-0.0749


### 2.3.

Compare the "level" factor beta for each of the two treasuries with the average  duration for each bond as reported in the data set.

* How closely does the average duration for a bond compare to its "level" beta?
* What do you conclude about the usefulness of mathematical duration vs regression sensitivities?

In [63]:
print(f'Mean duration:')
display(duration.mean())

Mean duration:


207391    6.9237
207392   19.9032
dtype: float64

The regression approach gets the right sensitivity on average (across the timeseries). But this unconditional estimate would be a poor guide to the duration at any particular day.

Note that the quantiles of the duration over this period range substantially from the average, which is what is estimated in the regression.

### 2.4.

In the duration-hedged trade of `Homework 2, Section 2`, was the that trade was long or short this slope factor? 

Do you think the slope factor exposure had a large impact on the trade?

No new analysis needed, just draw a conclusion from the estimates above along with the trade construction in `HW 2, Sec 2`.

***

# 3 Calculating Duration Via Individual Cashflows

## *Optional, not submitted*

Use the data file `../data/treasury_quotes_2024-10-31.xlsx`.

### 3.1 
Set up the cashflow matrix. 

### 3.2
Extract the Nelson-Siegel spot discount curve, as you did in `Homework 1`.

### 3.3
For each treasury issue, calculate the duration as the weighted average of the (discounted!) cashflow maturity.

Report the summary statistics of the durations. (Use `.describe()` from pandas.)

### 3.4
How close are your duration estimates to the imputed durations given in the data source?

Report the summary statistics of the imputed durations minus your calculated durations from above.

### 3.5
Continue using your assumed discount rates of `4.5`\% to calculate the convexity of each issue.

Report the summary statistics of these convexity calculations.

***